In [1]:
import cv2
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import random
import numpy as np
from PIL import Image


In [ ]:

# Step 1: Load the .mov video file
video_path = '../Screen Recording 2024-03-07 at 10.25.19 AM.mov'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
random_frame_number = random.randint(0, frame_count - 1)

# Step 2: Sample a random frame from the video

frames = []

for f in range(frame_count//50):
    if f%10==0:
        print(f,"/",frame_count)
    #cap.set(cv2.CAP_PROP_POS_FRAMES, f)
    success, frame = cap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    original_height, original_width = frame.shape[:2]

    # Calculate new dimensions (half)
    new_width = original_width // 2
    new_height = original_height // 2
    
    # Resize the frame to half of the original dimensions
    resized_frame = cv2.resize(frame_rgb, (new_width, new_height), interpolation=cv2.INTER_AREA)

    if success:
        frames.append(resized_frame)
    

In [159]:
video_path = '../Screen Recording 2024-03-07 at 10.25.19 AM.mov'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Step 2: Sample a random frame from the video

frames = []
frame_number=0
# Loop through the frames, skipping every 30 frames
while frame_number < total_frames:
    print(frame_number, "/", total_frames)
    # Set the video capture object to the current frame number
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # Read the frame
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if ret:
        # Convert the frame to RGB color space
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get the original dimensions of the frame
        original_height, original_width = frame.shape[:2]

        # Calculate new dimensions (half)
        new_width = original_width // 4
        new_height = original_height // 4

        # Resize the frame to half of the original dimensions
        resized_frame = cv2.resize(frame_rgb, (new_width, new_height), interpolation=cv2.INTER_AREA)

        # Append the resized frame to the list
        frames.append(resized_frame)
    else:
        break

    # Update the frame number to skip every 30 frames
    frame_number += 15

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()

0 / 8649
15 / 8649
30 / 8649
45 / 8649
60 / 8649
75 / 8649
90 / 8649
105 / 8649
120 / 8649
135 / 8649
150 / 8649
165 / 8649
180 / 8649
195 / 8649
210 / 8649
225 / 8649
240 / 8649
255 / 8649
270 / 8649
285 / 8649
300 / 8649
315 / 8649
330 / 8649
345 / 8649
360 / 8649
375 / 8649
390 / 8649
405 / 8649
420 / 8649
435 / 8649
450 / 8649
465 / 8649
480 / 8649
495 / 8649
510 / 8649
525 / 8649
540 / 8649
555 / 8649
570 / 8649
585 / 8649
600 / 8649
615 / 8649
630 / 8649
645 / 8649
660 / 8649
675 / 8649
690 / 8649
705 / 8649
720 / 8649
735 / 8649
750 / 8649
765 / 8649
780 / 8649
795 / 8649
810 / 8649
825 / 8649
840 / 8649
855 / 8649
870 / 8649
885 / 8649
900 / 8649
915 / 8649
930 / 8649
945 / 8649
960 / 8649
975 / 8649
990 / 8649
1005 / 8649
1020 / 8649
1035 / 8649
1050 / 8649
1065 / 8649
1080 / 8649
1095 / 8649
1110 / 8649
1125 / 8649
1140 / 8649
1155 / 8649
1170 / 8649
1185 / 8649
1200 / 8649
1215 / 8649
1230 / 8649
1245 / 8649
1260 / 8649
1275 / 8649
1290 / 8649
1305 / 8649
1320 / 8649
1335 / 

In [162]:
frames = np.array(frames)
np.save('frames.npy', frames)


In [2]:
frames=np.load('frames.npy')

In [3]:
import argparse
from fastsam import FastSAM, FastSAMPrompt 
import ast
import torch
from PIL import Image
from utils.tools import convert_box_xywh_to_xyxy


In [4]:
from PIL import Image
from scipy.spatial import cKDTree


def sm(im):
    Image.fromarray(im).show()

In [5]:

class args_init:
    def __init__(self):
        self.model_path = "./FastSAM.pt"
        self.img_path = "./images/imageForSegmentation.png"
        self.imgsz = 1024//2
        self.iou = 0.9
        self.text_prompt = None

        self.conf = 0.4
        self.output = "./output/"
        self.point_prompt="[[0,0]]"

        self.randomcolor = True


        self.point_label = "[0]"
        self.box_prompt = "[[0,0,0,0]]"
        self.better_quality = False
        self.device = torch.device(
            "cuda"
            #if torch.cuda.is_available()
            #else "mps"
            #if torch.backends.mps.is_available()
            #else "cpu"
        )
        self.retina = True
        self.withContours = False

args = args_init()

In [6]:

model = FastSAM(args.model_path)
args.point_prompt = ast.literal_eval(args.point_prompt)
args.box_prompt = convert_box_xywh_to_xyxy(ast.literal_eval(args.box_prompt))
args.point_label = ast.literal_eval(args.point_label)

bboxes = None
points = None
point_label = None



In [7]:
# load model

def run_SAM(frame):
    everything_results = model(
    frame,
    device=args.device,
    retina_masks=args.retina,
    imgsz=args.imgsz,
    conf=args.conf,
    iou=args.iou    
    )

    #prompt_process = FastSAMPrompt(frame, everything_results, device=args.device)
    #ann = prompt_process.everything_prompt()
    
    return everything_results[0].masks.data




In [8]:
import numpy as np
import cv2



#def render_com(mask):
#    cX, cY = get_center_of_mass(mask)
#    image_colored = cv2.cvtColor(mask*255.0, cv2.COLOR_GRAY2BGR)

#    image_colored = cv2.circle(image_colored, (cX, cY), 10, (0, 0, 255), -1)  # Red circle

#    sm(cv2.convertScaleAbs(image_colored))
        


In [9]:
run_SAM(Image.fromarray(frames[200]).convert("RGB"))


0: 288x512 51 objects, 170.0ms
Speed: 4.0ms preprocess, 170.0ms inference, 168.3ms postprocess per image at shape (1, 3, 512, 512)


tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [10]:
subset = 1000


In [ ]:
def get_center_of_mass(mask):
    mask = mask.astype('uint8')
    M = cv2.moments(mask)
    if M["m00"] == 0:
        return (0,0)
    
    moments=np.array([M[m] for m in ["m10", "m01"]])#, "m20", "m11", "m02", "m30", "m21", "m12", "m03"]])
    return moments/M["m00"]
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    return (cX, cY)


def get_moments(ann):
    annotation_locs = []
    for a in range(len(ann)):
        annotation_locs.append(get_center_of_mass(ann[a]))
    return np.array(annotation_locs)

moments = []
#for (f,ann) in enumerate(annons):
#    #moments.append([])
    

In [11]:
moments = []
annons = []
for (idx,f) in enumerate(range(len(frames))):
    ann = run_SAM(Image.fromarray(frames[f]).convert("RGB"))
    annotation_locs = []
    for a in range(len(ann)):

        avg_color = frames[f][ann[a]==1].sum(axis=0)/ann[a].sum()
        annotation_locs.append(list(get_center_of_mass(ann[a]))+list(avg_color))
        
    #print(annotation_locs)
    
    moments.append(np.array(annotation_locs))
    #moments.append(get_moments(ann))
    #print(ann)
    
    annons.append([a.cpu().detach().numpy() for a in ann])
    if idx%10==0:
        print(idx,"/",len(frames))
        
    if idx>subset:
        break



0: 288x512 65 objects, 85.0ms
Speed: 5.0ms preprocess, 85.0ms inference, 11.0ms postprocess per image at shape (1, 3, 512, 512)



0 / 575


0: 288x512 62 objects, 85.7ms
Speed: 2.0ms preprocess, 85.7ms inference, 12.0ms postprocess per image at shape (1, 3, 512, 512)

0: 288x512 62 objects, 85.1ms
Speed: 2.0ms preprocess, 85.1ms inference, 11.0ms postprocess per image at shape (1, 3, 512, 512)

0: 288x512 62 objects, 85.0ms
Speed: 2.0ms preprocess, 85.0ms inference, 11.0ms postprocess per image at shape (1, 3, 512, 512)

0: 288x512 62 objects, 85.0ms
Speed: 1.0ms preprocess, 85.0ms inference, 11.0ms postprocess per image at shape (1, 3, 512, 512)



KeyboardInterrupt: 

In [46]:
len(annons)

102

In [25]:
moments

[array([[     1287.7,      144.87,  1.6604e+06,  1.8703e+05,       26317,  2.1436e+09,  2.4184e+08,  3.4088e+07,  5.2975e+06],
        [     176.91,      711.04,       38531,  1.2618e+05,  5.0693e+05,  9.4927e+06,   2.759e+07,  9.0212e+07,  3.6236e+08],
        [     859.65,      460.08,  7.4481e+05,  3.9579e+05,  2.1197e+05,  6.5034e+08,   3.432e+08,  1.8248e+08,  9.7797e+07],
        [     1181.9,      51.472,  1.3975e+06,       60681,      3442.2,  1.6532e+09,  7.1569e+07,  4.0523e+06,  2.5995e+05],
        [     471.22,      451.03,  2.2371e+05,  2.1254e+05,  2.0377e+05,  1.0697e+08,   1.009e+08,  9.6025e+07,  9.2218e+07],
        [     542.28,      747.32,  2.9503e+05,  4.0521e+05,   5.587e+05,  1.6103e+08,  2.2043e+08,   3.029e+08,  4.1784e+08],
        [     346.65,      191.93,  1.2079e+05,       66594,       39098,    4.23e+07,  2.3223e+07,  1.3577e+07,  8.3787e+06],
        [     372.52,         577,  1.4002e+05,  2.1304e+05,  3.3642e+05,   5.309e+07,  7.9373e+07,  1.2313e+08

In [155]:
from scipy.spatial import cKDTree


In [156]:
frames_annotate = frames.copy()



In [157]:
trackers = moments[0]
last_seen = [0 for t in trackers]
colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for t in trackers]

for (idx,m) in enumerate(moments[1:]):
    
    tree = cKDTree(trackers)
    dists, neighbors = tree.query(m)
    
    canidate_updates = {}
    for (i,n) in enumerate(neighbors):
        if dists[i]<100 and last_seen[n]<12:
            if n not in canidate_updates:
                canidate_updates[n] = [i]
            else:
                canidate_updates[n].append(i)
        else:
            trackers=np.append(trackers,m[i].reshape(1,5),axis=0)
            last_seen.append(0)
            colors.append((random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)))

        last_seen[n] += 1
    
    for c in canidate_updates:
        last_seen[c] = 0
        
        min_idx = canidate_updates[c][np.argmin([dists[can] for can in canidate_updates[c]])]
        
        trackers[c] = m[min_idx]
        cX, cY = m[min_idx][:2]
        color = (m[min_idx][2],m[min_idx][3],m[min_idx][4])#colors[c]
        frames_annotate[idx] = cv2.circle(frames_annotate[idx], (int(cX), int(cY)), 10, color, -1)
        frames_annotate[idx] = cv2.circle(frames_annotate[idx], (int(cX), int(cY)), 10, colors[c], 4)



In [158]:
num_frames, height, width, channels = frames_annotate.shape

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec as needed
fps = 5.0  # Frames per second
video_writer = cv2.VideoWriter('output_video.mp4', fourcc, fps, (width, height))

# Loop through the frames and write them to the video
for frame in frames_annotate[:100]:
    # Convert the frame from NumPy array to OpenCV format
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # Write the frame to the video
    video_writer.write(frame)

# Release the video writer object
video_writer.release()